In [ ]:
!pip install accelerate -U
!pip install datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=5e4c8b9b38cf94149d4b033edca4939b7db0b77068ea88718ae817be2b348f8e
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import torch
import numpy as np
import transformers, datasets, evaluate, seqeval

from datasets import load_dataset, load_metric
from transformers import AutoModelForTokenClassification, RobertaTokenizerFast, DataCollatorForTokenClassification

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
data_set_test = load_dataset('Babelscape/multinerd',split='test')

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
en_data_test_A = data_set_test.filter(lambda x: x['lang']=='en')
en_data_test_B = data_set_test.filter(lambda x: x['lang']=='en' and (set(x['ner_tags']) & {1,2,3,4,5,6,7,8,13,14}))

Filter:   0%|          | 0/335986 [00:00<?, ? examples/s]

Filter:   0%|          | 0/335986 [00:00<?, ? examples/s]

In [ ]:
tokenizer_A = RobertaTokenizerFast.from_pretrained('Saketh/entity-recognition-general-sota-v1-finetuned-ner',add_prefix_space=True,padding=True, truncation=True)

tokenizer_B = RobertaTokenizerFast.from_pretrained('Saketh/entity-recognition-general-sota-v1-finetuned-ner-X',add_prefix_space=True,padding=True, truncation=True)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels_A(examples):
    tokenized_inputs = tokenizer_A(examples["tokens"],padding=True, truncation=True,is_split_into_words=True,return_tensors="pt")

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Setting the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
def tokenize_and_align_labels_B(examples):
    tokenized_inputs = tokenizer_B(examples["tokens"],padding=True, truncation=True,is_split_into_words=True,return_tensors="pt")
    req_label_list = [0,1,2,3,4,5,6,7,8,13,14]

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        #print(label)
        label = [x if x in req_label_list else 0 for x in label]       # keeping only the required tags and replaceing zero for others
        label = [9 if x==13 else 10 if x==14 else x for x in label ]   # changing tags from 13,14 to 9,10 to be in serial order
        examples["ner_tags"][i] = label
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Setting the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    tokenized_inputs["ner_tags"] = examples["ner_tags"]
    #print(examples)
    return tokenized_inputs

In [ ]:
mapped_data_test_A = en_data_test_A.map(tokenize_and_align_labels_A,batched=True,batch_size=1)

mapped_data_test_B = en_data_test_B.map(tokenize_and_align_labels_B,batched=True,batch_size=1)

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

Map:   0%|          | 0/29512 [00:00<?, ? examples/s]

In [ ]:
mapped_data_test_A.set_format(type="torch", columns=["input_ids","attention_mask"], output_all_columns=True)
mapped_data_test_B.set_format(type="torch", columns=["input_ids","attention_mask"], output_all_columns=True)

In [ ]:
labels2ids_A = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-ANIM": 7,
    "I-ANIM": 8,
    "B-BIO": 9,
    "I-BIO": 10,
    "B-CEL": 11,
    "I-CEL": 12,
    "B-DIS": 13,
    "I-DIS": 14,
    "B-EVE": 15,
    "I-EVE": 16,
    "B-FOOD": 17,
    "I-FOOD": 18,
    "B-INST": 19,
    "I-INST": 20,
    "B-MEDIA": 21,
    "I-MEDIA": 22,
    "B-MYTH": 23,
    "I-MYTH": 24,
    "B-PLANT": 25,
    "I-PLANT": 26,
    "B-TIME": 27,
    "I-TIME": 28,
    "B-VEHI": 29,
    "I-VEHI": 30,
  }

ids2labels_A = {value:key for key,value in labels2ids_A.items()}
label_list_A = [key for key,value in labels2ids_A.items()]

In [ ]:
labels2ids_B = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-ANIM": 7,
    "I-ANIM": 8,
    "B-DIS": 9,
    "I-DIS": 10
  }

ids2labels_B = {value:key for key,value in labels2ids_B.items()}
label_list_B = [key for key,value in labels2ids_B.items()]

In [ ]:
model_A = AutoModelForTokenClassification.from_pretrained('Saketh/entity-recognition-general-sota-v1-finetuned-ner',num_labels=31, id2label=ids2labels_A, label2id=labels2ids_A)

model_B = AutoModelForTokenClassification.from_pretrained('Saketh/entity-recognition-general-sota-v1-finetuned-ner-X',num_labels=11, id2label=ids2labels_B, label2id=labels2ids_B)

config.json:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

In [ ]:
data_collator_A = DataCollatorForTokenClassification(tokenizer_A)
data_collator_B = DataCollatorForTokenClassification(tokenizer_B)

In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
def compute_metrics_A(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [ [label_list_A[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    true_labels = [ [label_list_A[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

def compute_metrics_B(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [ [label_list_B[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    true_labels = [ [label_list_B[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer_A = Trainer(
    model_A,
    # args,
    # train_dataset=mapped_data,
    # eval_dataset=mapped_data_v,
    data_collator=data_collator_A,
    tokenizer=tokenizer_A,
    compute_metrics=compute_metrics_A
)

In [ ]:
trainer_B = Trainer(
    model_B,
    # args,
    # train_dataset=mapped_data,
    # eval_dataset=mapped_data_v,
    data_collator=data_collator_B,
    tokenizer=tokenizer_B,
    compute_metrics=compute_metrics_B
)

In [ ]:
trainer_A.evaluate(eval_dataset=mapped_data_test_A)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.02707073651254177,
 'eval_precision': 0.9378337879893957,
 'eval_recall': 0.9518034704620784,
 'eval_f1': 0.9447669917943954,
 'eval_accuracy': 0.9901678553418342,
 'eval_runtime': 133.0665,
 'eval_samples_per_second': 247.305,
 'eval_steps_per_second': 30.917}

In [ ]:
trainer_B.evaluate(eval_dataset=mapped_data_test_B)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.017866812646389008,
 'eval_precision': 0.9557654500384648,
 'eval_recall': 0.9739558381603589,
 'eval_f1': 0.9647749088634354,
 'eval_accuracy': 0.9931328078645237,
 'eval_runtime': 109.6919,
 'eval_samples_per_second': 269.045,
 'eval_steps_per_second': 33.631}